In [1]:
# To Match Karst to Tile
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta

from shapely.geometry import Point, Polygon
import geopandas as gpd

# my script
from w210_attribute_library_scale import tilekey, haversine_distance, withinstates

datdir = "../data/"
attrs = "../attrs/"
modeld = "../model/"

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Read Soil Data

In [2]:
soild = pd.read_csv(datdir+'FL_soil_byCounty.csv')
print(len(soild))

67


In [7]:
soild.columns

Index(['County', 'Center Latitude', 'Center Longitude', 'Calcium Carbonate',
       'Gypsum', 'Soil Health Organic Matter', 'Percent Clay', 'Percent Sand',
       'Percent Silt', 'Available Water Storage'],
      dtype='object')

## Read Tiles

In [3]:
ftileslarge = 'scale_up_F_satellite_ws_attr.csv'
dftiles_large = pd.read_csv(attrs+ftileslarge)
# dftiles_large["Key"] = dftiles_large.apply(lambda row: tilekey(row), axis=1)
# dftiles_large = dftiles_large[["Key", "lon", "lat", "County", "CountyFP"]]
# dftiles_large.columns = ["Key", "lon_t", "lat_t", "County", "CountyFp"]

print(len(dftiles_large))
dftiles_large.head(2)

407463


,Key,Key_y,date_ws,name,lon_t,lat_t,lon_w,lat_w,Distance,rolling_7_precip,...,y2_mean_prc,y2_max_prc,y2_mean_tmp,y2_max_tmp,y2_min_tmp,y3_mean_prc,y3_max_prc,y3_mean_tmp,y3_max_tmp,y3_min_tmp
0,033_174,720383_53847,2021-06-01,WHITING FIELD NAVAL AIR STATI,-87.624327,30.880381,-87.023,30.704,37.756111,0.63,...,0.093381,2.82,69.399178,85.9,35.5,0.173123,3.03,67.569589,85.3,37.6
1,033_206,720383_53847,2021-06-01,WHITING FIELD NAVAL AIR STATI,-87.618608,30.897536,-87.023,30.704,37.835130,0.63,...,0.093381,2.82,69.399178,85.9,35.5,0.173123,3.03,67.569589,85.3,37.6


In [4]:
# Cross Merge Dataframes
dffinal = pd.merge(dftiles_large,soild, how="cross")
print(len(dffinal))

27300021


In [5]:
dffinal.columns

Index(['Key', 'Key_y', 'date_ws', 'name', 'lon_t', 'lat_t', 'lon_w', 'lat_w',
       'Distance', 'rolling_7_precip', 'rolling_15_precip',
       'rolling_30_precip', 'rolling_60_precip', 'rolling_90_precip',
       'y1_mean_prc', 'y1_max_prc', 'y1_mean_tmp', 'y1_max_tmp', 'y1_min_tmp',
       'y2_mean_prc', 'y2_max_prc', 'y2_mean_tmp', 'y2_max_tmp', 'y2_min_tmp',
       'y3_mean_prc', 'y3_max_prc', 'y3_mean_tmp', 'y3_max_tmp', 'y3_min_tmp',
       'County', 'Center Latitude', 'Center Longitude', 'Calcium Carbonate',
       'Gypsum', 'Soil Health Organic Matter', 'Percent Clay', 'Percent Sand',
       'Percent Silt', 'Available Water Storage'],
      dtype='object')

In [8]:
# Get Distance
dffinal['Distance'] = dffinal.apply(lambda row: 
                                    haversine_distance(row['lat_t'], row['lon_t'], 
                                                       row['Center Latitude'], 
                                                       row['Center Longitude'], 
                                                       earth_radius=3963.19), axis=1)

## Find the Minimum Distances within Tile and Soil Point

`df.groupby('Company')['MPG'].agg('min')`  
`df.groupby('Company')[['MPG', 'EngineSize']].agg('min')`

**Reference:**
https://datascienceparichay.com/article/pandas-groupby-minimum/

In [9]:
dfmin1 = dffinal.groupby(['Key'])['Distance'].min().to_frame()

keysL = list(dfmin1.index)
minD = list(dfmin1['Distance'])
dfF1 = dffinal[((dffinal['Key'].isin(keysL)) &  (dffinal['Distance'].isin(minD)))]
len(dfF1)

dfF1.to_csv(attrs+"scale_up_F_satellite_ws_soil_attr.csv", index=False)